In [1]:
import numpy as np

def save_data(data, labels):
    file_name = 'Data\\training_data_test.npz'
    np.savez_compressed(file_name,data=data,labels=labels)

In [2]:
data = []

for n in range(200):
    print("Generating image " + str(n) + "...")
    data.append([])
    for i in range(5):
        data[n].append([])
        for j in range(270):
            data[n][i].append([])
            for k in range(480):
                data[n][i][j].append([])
                for l in range(3):
                    data[n][i][j][k].append(np.float32(np.random.randint(255)/255))
                    
data = np.array(data)

print(data.shape)

Generating image 0...
Generating image 1...
Generating image 2...
Generating image 3...
Generating image 4...
Generating image 5...
Generating image 6...
Generating image 7...
Generating image 8...
Generating image 9...
Generating image 10...
Generating image 11...
Generating image 12...
Generating image 13...
Generating image 14...
Generating image 15...
Generating image 16...
Generating image 17...
Generating image 18...
Generating image 19...
Generating image 20...
Generating image 21...
Generating image 22...
Generating image 23...
Generating image 24...
Generating image 25...
Generating image 26...
Generating image 27...
Generating image 28...
Generating image 29...
Generating image 30...
Generating image 31...
Generating image 32...
Generating image 33...
Generating image 34...
Generating image 35...
Generating image 36...
Generating image 37...
Generating image 38...
Generating image 39...
Generating image 40...
Generating image 41...
Generating image 42...
Generating image 43..

In [3]:
labels = []

for n in range(200):
    print("Creating label " + str(n) + "...")
    labels.append([0,0,0,0,0,0,0,0,0])
    i = np.random.choice(list(range(9)))
    labels[n][i] = 1

    
labels = np.array(labels)
print(labels.shape)

Creating label 0...
Creating label 1...
Creating label 2...
Creating label 3...
Creating label 4...
Creating label 5...
Creating label 6...
Creating label 7...
Creating label 8...
Creating label 9...
Creating label 10...
Creating label 11...
Creating label 12...
Creating label 13...
Creating label 14...
Creating label 15...
Creating label 16...
Creating label 17...
Creating label 18...
Creating label 19...
Creating label 20...
Creating label 21...
Creating label 22...
Creating label 23...
Creating label 24...
Creating label 25...
Creating label 26...
Creating label 27...
Creating label 28...
Creating label 29...
Creating label 30...
Creating label 31...
Creating label 32...
Creating label 33...
Creating label 34...
Creating label 35...
Creating label 36...
Creating label 37...
Creating label 38...
Creating label 39...
Creating label 40...
Creating label 41...
Creating label 42...
Creating label 43...
Creating label 44...
Creating label 45...
Creating label 46...
Creating label 47...
Cr

In [4]:
save_data(data, labels)